---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [18]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [19]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
# new_states_dict = {}
# for key,value in states.items():
#     new_states_dict[value] = key
# #new_states_dict

In [20]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Procesando .txt
    Ucitys = open("university_towns.txt", 'r').read().split('\n')

    # Bucle para crear listas en formato adecuado
    L = []
    for i in range(len(Ucitys)): # range(195, 215): 
        # Se detecta un nombre de estado
        if Ucitys[i][-6:] == '[edit]':
            last_state = Ucitys[i][:-6]
            continue
        # Se empareja ciudad con estados
        locate_symbol = Ucitys[i].find('(')-1
        if locate_symbol == -2:
            town_name = Ucitys[i]
        else:
            town_name = Ucitys[i][:locate_symbol]
        #print(town_name)
        L.append([last_state, town_name])

    df = pd.DataFrame(L[:-1], columns = ["State", "RegionName"])
    # df['State'] = df['State'].apply(lambda x: new_states_dict[x])
    return df

#get_list_of_university_towns() #.iloc[90:120]

In [21]:
# import re
# import pandas as pd
# import numpy as np
# # list of unique states
# stateStr = """
# Ohio, Kentucky, American Samoa, Nevada, Wyoming
# ,National, Alabama, Maryland, Alaska, Utah
# ,Oregon, Montana, Illinois, Tennessee, District of Columbia
# ,Vermont, Idaho, Arkansas, Maine, Washington
# ,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
# ,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
# ,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
# ,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
# ,New York, Nebraska, Oklahoma, Florida, California
# ,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
# ,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
# ,North Dakota, Virginia
# """
# #list of regionName entries string length
# regNmLenStr = """
# 06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
# 06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
# 13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
# 11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
# 11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
# 08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
# 18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
# 16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
# 09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
# 06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
# 10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
# 08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
# 05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
# 06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
# 09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
# 16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
# 11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
# 09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
# 07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
# 07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
# 10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
# 08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
# 12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
# 09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
# 09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
# 10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
# """

# df = get_list_of_university_towns()

# cols = ["State", "RegionName"]

# print('Shape test: ', "Passed" if df.shape ==
#       (517, 2) else 'Failed')
# print('Index test: ',
#       "Passed" if df.index.tolist() == list(range(517))
#       else 'Failed')

# print('Column test: ',
#       "Passed" if df.columns.tolist() == cols else 'Failed')
# print('\\n test: ',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\n')) or any(df[cols[1]].str.contains('\n'))
#       else 'Passed')
# print('Trailing whitespace test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\s+$')) or any(df[cols[1]].str.contains(
#               '\s+$'))
#       else 'Passed')
# print('"(" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\(')) or any(df[cols[1]].str.contains(
#               '\('))
#       else 'Passed')
# print('"[" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\[')) or any(df[cols[1]].str.contains(
#               '\]'))
#       else 'Passed')

# states_vlist = [st.strip() for st in stateStr.split(',')]

# mismatchedStates = df[~df['State'].isin(
#     states_vlist)].loc[:, 'State'].unique()
# print('State test: ', "Passed" if len(
#     mismatchedStates) == 0 else "Failed")
# if len(mismatchedStates) > 0:
#     print()
#     print('The following states failed the equality test:')
#     print()
#     print('\n'.join(mismatchedStates))

# df['expected_length'] = [int(s.strip())
#                          for s in regNmLenStr.split(',')
#                          if s.strip().isdigit()]
# regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
#     :, ['RegionName', 'expected_length']]
# regDiff['actual_length'] = regDiff['RegionName'].str.len()
# print('RegionName test: ', "Passed" if len(regDiff) ==
#       0 else ' \nMismatching regionNames\n {}'.format(regDiff))


A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

A recession bottom is the quarter within a recession which had the lowest GDP.

In [22]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_df = pd.read_excel('gdplev.xls', usecols = ['Unnamed: 4', 'Unnamed: 6'], header = 6) # , index_col = 4, header = 1)
    GDP_df.columns = ["Quarter","GDP in billions of chained 2009 dollars"]
    GDP_df = GDP_df[GDP_df.Quarter >= "2000q1"].reset_index()
    GDP_df.drop('index', axis = 1, inplace = True)
    # Variables... 
    gdp_2 = 0
    gdp_1 = 0
    i = 0
    for gdp in GDP_df['GDP in billions of chained 2009 dollars']:
        gdp_0 = gdp
        if gdp_2 > gdp_1 > gdp_0:
            return(GDP_df['Quarter'].iloc[i-1]) #
            #break
        else:
            gdp_2 = gdp_1
            gdp_1 = gdp_0
            i += 1
    return None

#get_recession_start()

In [23]:
# import matplotlib.pyplot as plt

# plt.plot(GDP_df['GDP in billions of chained 2009 dollars'][30:45], '.')
# plt.show()

In [24]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    GDP_df = pd.read_excel('gdplev.xls', usecols = ['Unnamed: 4', 'Unnamed: 6'], header = 6) # , index_col = 4, header = 1)
    GDP_df.columns = ["Quarter","GDP in billions of chained 2009 dollars"]
    GDP_df = GDP_df[GDP_df.Quarter >= "2000q1"].reset_index()
    GDP_df.drop('index', axis = 1, inplace = True)
    
    # Starting from recession start
    GDP_df = GDP_df[GDP_df.Quarter >= '2008q3'] # get_recession_start() ]
    
    gdp_2 = 0
    gdp_1 = 0
    i = 0
    for gdp in GDP_df['GDP in billions of chained 2009 dollars']:
        gdp_0 = gdp
        if gdp_0 > gdp_1 > gdp_2:
            return(GDP_df['Quarter'].iloc[i])
        else:
            gdp_2 = gdp_1
            gdp_1 = gdp_0
            i += 1
    return None

#get_recession_end()

In [25]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    # Getting DF
    GDP_df = pd.read_excel('gdplev.xls', usecols = ['Unnamed: 4', 'Unnamed: 6'], header = 6) # , index_col = 4, header = 1)
    GDP_df.columns = ["Quarter","GDP in billions of chained 2009 dollars"]
    GDP_df = GDP_df[GDP_df.Quarter >= "2000q1"].reset_index()
    GDP_df.drop('index', axis = 1, inplace = True)
    # Starting from recession start up until recession ends
    GDP_df = GDP_df[(GDP_df.Quarter >= '2008q3') & (GDP_df.Quarter <= '2009q4')]
    # Arg min on GDP, then retrieve Quarter
    Bottom = GDP_df.Quarter.loc[ GDP_df['GDP in billions of chained 2009 dollars'].idxmin() ]
    return Bottom

#get_recession_bottom()

In [26]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    houses_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    # Se eliminan columnas que no quiero
    houses_df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis = 1, inplace = True)
    houses_df.drop([c for c in houses_df.columns if c < '2000-01'], axis = 1, inplace = True)
    houses_df['State'] = houses_df['State'].apply(lambda x: states[x])
    # Setting multiindex
    houses_df = houses_df.set_index(['State', 'RegionName'])
    houses_df.sort_index(inplace = True)

    # Lista todas las columnas númericas
    Year_months = list( houses_df.columns )

    # Generador de quarters
    Quarters = []
    for year in range(2000, 2017):
        for q in range(1,5):
            Quarters.append(str(year) + 'q' + str(q))        
    # Se quita ultimo quarter
    Quarters = Quarters[:-1]

    Quarters_months = []
    # Generador de meses asociados a quarter
    for q in Quarters:
        Quarters_months.append([q, list(map(lambda x: q[:4] + '-' + str(x).zfill(2), list((int(q[-1]) - 1)*3 + np.arange(1,4))))])

    # Se modifica caso especial del último cuarto
    Quarters_months[-1] = ['2016q3', ['2016-07', '2016-08']] 

    # Se crean nuevas columnas con promedio y se eliminan otras
    for q in Quarters_months:
        cols = np.array(q[1:][0])
        houses_df[q[0]] = houses_df[cols].mean(axis = 1)
        houses_df.drop(cols, axis = 1, inplace = True)
    
    return houses_df

# convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

In [27]:
# # Data of the decline of growth of housing prices between the recesion start and the recession bottom. 
# Housing_data = convert_housing_data_to_quarters()
# quarter_before_recession = '2008q2'
# recession_bottom = get_recession_bottom()
# not_useful_quaters = [x for x in Housing_data.columns if (x < quarter_before_recession) or (x > recession_bottom)]


# # Eliminando lo que no nos importa...
# Housing_data.drop(not_useful_quaters, axis = 1, inplace = True)
                  
# Housing_data['Price_ratio'] = Housing_data[quarter_before_recession] / Housing_data[recession_bottom]

# # Se eliminan donde price_ratio es nan...
# Housing_data = Housing_data[Housing_data['Price_ratio'].notnull()]

# Housing_data.head()
# #Housing_data['Price_ratio'].describe()

In [28]:
# # Accediendo a housing data de uni_towns..
# Array_1 = np.array(uni_towns['State'])
# Array_2 = np.array(uni_towns['RegionName'])
# I = pd.MultiIndex.from_arrays([Array_1, Array_2], names = ('State', 'RegionName'))

# Housing_data_uni_towns = Housing_data.loc[I]
# Housing_data_uni_towns.dropna(how = 'all', inplace = True)
# Housing_data_uni_towns

In [29]:
# Housing_data_not_uni_towns = Housing_data.drop(I, axis = 0)
# Housing_data_not_uni_towns

In [30]:
# # Verificanco división
# print(Housing_data.shape)
# print(Housing_data_uni_towns.shape)
# print(Housing_data_not_uni_towns.shape)

In [31]:
#Housing_data_uni_towns['Price_ratio'].mean()

In [32]:
#Housing_data_not_uni_towns['Price_ratio'].mean()

In [33]:
# s,p = ttest_ind(Housing_data_uni_towns['Price_ratio'], Housing_data_not_uni_towns['Price_ratio'] )

In [35]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # Data of the decline of growth of housing prices between the recesion start and the recession bottom. 
    Housing_data = convert_housing_data_to_quarters()
    quarter_before_recession = '2008q2'
    recession_bottom = get_recession_bottom()
    not_useful_quaters = [x for x in Housing_data.columns if (x < quarter_before_recession) or (x > recession_bottom)]

    # Eliminando lo que no nos importa...
    Housing_data.drop(not_useful_quaters, axis = 1, inplace = True)
    Housing_data['Price_ratio'] = Housing_data[quarter_before_recession] / Housing_data[recession_bottom]

    # Se eliminan donde price_ratio es nan...
    Housing_data = Housing_data[Housing_data['Price_ratio'].notnull()]

    # Accediendo a housing data de uni_towns..
    uni_towns = get_list_of_university_towns()
    Array_1 = np.array(uni_towns['State'])
    Array_2 = np.array(uni_towns['RegionName'])
    I = pd.MultiIndex.from_arrays([Array_1, Array_2], names = ('State', 'RegionName'))
    # Accediendo a multiindex creado
    Housing_data_uni_towns = Housing_data.loc[I]
    # Elimnando nan
    Housing_data_uni_towns.dropna(how = 'all', inplace = True)
    # No university town
    Housing_data_not_uni_towns = Housing_data.drop(I, axis = 0)
    
    # Ttest
    s,p = ttest_ind(Housing_data_uni_towns['Price_ratio'], Housing_data_not_uni_towns['Price_ratio'])
    different = p < 0.01
    better = "university town" if different else "non-university town"
    return different, p, better

run_ttest()

(True, 0.002724063704761164, 'university town')